In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


In [2]:
#df = pd.read_csv('../input/train.csv')
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# Fill missing fields with columns means
df = df.fillna(df.mean())
df = df.fillna('N/A')

# Fill missing fields with columns means
df_test = df_test.fillna(df.mean())
df_test = df_test.fillna('N/A')

print(df.head())
print(df.describe())
print(df.dtypes)



   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   N/A        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   N/A        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   N/A        S  
  

# Labeling

In [3]:
labels = ['Sex', 'Embarked']
les = {}

for l in labels:
    print('labeling ' + l)
    les[l] = LabelEncoder()
    les[l].fit(df[l])
    tr = les[l].transform(df[l]) 
    df.loc[:, l + '_feat'] = pd.Series(tr, index=df.index)

    tr_test = les[l].transform(df_test[l]) 
    df_test.loc[:, l + '_feat'] = pd.Series(tr_test, index=df_test.index)


# le.classes_
print(df.head())

labeling Sex
labeling Embarked
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  Sex_feat  Embarked_feat  
0      0         A/5 21171   7.2500   N/A        S         1              3  
1      0          PC 17599  71.2833   C85        C         0              0  
2      0  STON/O2. 3101282   7.9250   N/A        S     

# Preparing datasets

In [ ]:
X_train = df.drop(labels, 1) \
    .drop('Survived', 1) \
    .drop('Cabin', 1) \
    .drop('Ticket', 1) \
    .drop('Name', 1) \
    .drop('PassengerId', 1)
y_train = df['Survived']

X_test = df_test.drop(labels, 1) \
    .drop('Cabin', 1) \
    .drop('Ticket', 1) \
    .drop('Name', 1) \
    .drop('PassengerId', 1)


print(X_train.head())
print(y_train.shape)
print(X_test.head())


   Pclass   Age  SibSp  Parch     Fare  Sex_feat  Embarked_feat
0       3  22.0      1      0   7.2500         1              3
1       1  38.0      1      0  71.2833         0              0
2       3  26.0      0      0   7.9250         0              3
3       1  35.0      1      0  53.1000         0              3
4       3  35.0      0      0   8.0500         1              3
(891,)
   Pclass   Age  SibSp  Parch     Fare  Sex_feat  Embarked_feat
0       3  34.5      0      0   7.8292         1              2
1       3  47.0      1      0   7.0000         0              3
2       2  62.0      0      0   9.6875         1              2
3       3  27.0      0      0   8.6625         1              3
4       3  22.0      1      1  12.2875         0              3


# Starting training set

In [ ]:

C = 1.0  # SVM regularization parameter
svc_linear = svm.SVC(kernel='linear', C=C).fit(X_train, y_train)


In [ ]:
svc_linear.score(X_train, y_train)

# Testing on the test set

In [ ]:
pred = svc_linear.predict(X_test)
print(pred.shape)

df_test['Survived'] = pd.Series(pred)
sub = df_test[['PassengerId','Survived']]
print(sub.head())
print(sub.tail())

sub.to_csv('submission_linear.csv', index=False)


In [ ]:

C = 1.0  # SVM regularization parameter
svc_rbf = svm.SVC(kernel='rbf', C=C).fit(X_train, y_train)
svc_rbf.score(X_train, y_train)

pred = svc_rbf.predict(X_test)
print(pred.shape)

df_test['Survived'] = pd.Series(pred)
sub = df_test[['PassengerId','Survived']]
print(sub.head())
print(sub.tail())


sub.to_csv('submission_rbf.csv', index=False)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 50)
forest.fit(X_train, y_train)

In [ ]:
forest.score(X_train, y_train)

In [ ]:
pred = forest.predict(X_test)
print(pred.shape)

df_test['Survived'] = pd.Series(pred)
sub = df_test[['PassengerId','Survived']]
print(sub.head())
print(sub.tail())


sub.to_csv('submission_forest.csv', index=False)


In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)


In [ ]:
lr.score(X_train, y_train)

In [ ]:
pred = lr.predict(X_test)
print(pred.shape)

df_test['Survived'] = pd.Series(pred)
sub = df_test[['PassengerId','Survived']]
print(sub.head())
print(sub.tail())


sub.to_csv('submission_lr.csv', index=False)
